In [6]:
import fasttext
import numpy as np
from scipy.spatial import distance
from scipy import stats
import json
import io

In [9]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(list(map(float, tokens[1:])))
    return data
# hin_embeddings_backup = fasttext.load_model("hin_embeddings/hin.embeddings.300.bin")
hin_embeddings = fasttext.load_model("hin_embeddings/cc.hi.300.bin")
# hin_embeddings = load_vectors("hin_embeddings/hin.crosslingual.300.vec")
# hin_embeddings = load_vectors("hin_embeddings/hin.cc.crosslingual.300.vec")

KeyboardInterrupt: 

In [8]:
mr_embeddings = fasttext.load_model('mr_embeddings/cc.mr.300.bin')
# mr_embeddings = fasttext.load_model("mr_embeddings/mr.embeddings.300.bin")
# mr_embeddings = fasttext.load_model("mr_embeddings/mr.embeddings.100.bin")
# mr_embeddings = load_vectors("mr_embeddings/mr.cc.crosslingual.300.vec")

In [16]:
mr_embeddings_backup = fasttext.load_model("mr_embeddings/mr.embeddings.300.bin")

In [12]:
x = map(float, [1,2])
np.asarray(list(x))

array([1., 2.])

In [11]:
def get_model_judgment(word_pair):
    (word1, word2) = word_pair
    try:
        a1 = hin_embeddings[word1]
        
        
    except KeyError:
        print(word1)
        a1 = mr_embeddings_backup[word1]
        
    try:
        b1 = hin_embeddings[word2]
    except KeyError:
        print(word2)
        b1 = hin_embeddings_backup[word2]
        
    return (1 - distance.cosine(a1, b1))
        

In [ ]:
def get_model_judgments(word_pairs):
    judgments = [get_model_judgment(word_pair) for word_pair in word_pairs]
    return judgments

In [ ]:
# Get word sim dataset
# Process word sim dataset
def get_word_sim_dataset(path):
    word_sim = open(path, "r").read().split("\n")
    word_pairs = [(pair.split("\t")[0], pair.split("\t")[1]) for pair in word_sim]
    judgments = [float(pair.split("\t")[2]) for pair in word_sim]
    return word_pairs, judgments

In [21]:
# 300 fasttext (trained on segmented)

word_pairs, human_judgments = get_word_sim_dataset("evaluation/hin.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

SpearmanrResult(correlation=0.5072718045006983, pvalue=9.025206876918802e-17)


In [15]:
#cc 300 fasttext (downloaded)

word_pairs, human_judgments = get_word_sim_dataset("evaluation/hin.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

SpearmanrResult(correlation=0.5750679275760561, pvalue=4.355153712018086e-22)


In [36]:
#CROSSLINGUAL with 300 fasttext (trained on segmented) as backup
word_pairs, human_judgments = get_word_sim_dataset("evaluation/hin.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(len(human_judgments))
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

235
SpearmanrResult(correlation=0.4889169385095757, pvalue=1.590491772145264e-15)


In [10]:
#Marathi cc 300 pretrained on mr cognates
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.cognates2.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

(8.5, 0.4548126757144928, ('संगणक', 'इंटरनेट'))
(6.75, 0.24530918896198273, ('विमान', 'कार'))
(6.625, 0.17394493520259857, ('रेल्वे', 'कार'))
(9.0, 0.19466328620910645, ('टेलिफोन', 'संवाद'))
(7.0, 0.3299368917942047, ('दूरदर्शन', 'रेडिओ'))
(8.0, 0.1975981742143631, ('मीडिया', 'रेडिओ'))
(6.375, 0.37264373898506165, ('डॉक्टर', 'परिचारिका'))
(4.0, 0.2736801505088806, ('प्राध्यापक', 'डॉक्टर'))
(6.0, 0.387312650680542, ('विद्यार्थी', 'प्राध्यापक'))
(4.375, 0.13760370016098022, ('स्मार्ट', 'विद्यार्थी'))
(3.0, 0.08245529979467392, ('स्मार्ट', 'मूर्ख'))
(6.375, 0.29583266377449036, ('पुस्तक', 'ग्रंथालय'))
(8.5, 0.24927856028079987, ('बँक', 'पैसा'))
(2.375, 0.0989447683095932, ('पवित्र', 'सेक्स'))
(6.375, 0.21721553802490234, ('टेनिस', 'रॅकेट'))
(8.375, 0.26931485533714294, ('अराफात', 'दहशतवादी'))
(0.0, 0.15169468522071838, ('अराफात', 'जॅक्सन'))
(8.5, 0.2683676779270172, ('कायदा', 'वकील'))
(1.0, 0.014892476610839367, ('पेय', 'गाडी'))
(0.0, 0.010776029899716377, ('पेय', 'कान'))
(6.5, 0.14536872

In [50]:
#Hindi cc 300 pretrained on mr cognates
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.cognates.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

(7.375, 0.2626831829547882, ('पुस्तक', 'कागद'))
(8.5, 0.3290926516056061, ('संगणक', 'इंटरनेट'))
(6.75, 0.4093678891658783, ('विमान', 'कार'))
(6.625, 0.19273625314235687, ('रेल्वे', 'कार'))
(9.0, 0.14239197969436646, ('टेलिफोन', 'संवाद'))
(7.0, 0.4051612317562103, ('दूरदर्शन', 'रेडिओ'))
(8.0, 0.3028254806995392, ('मीडिया', 'रेडिओ'))
(6.375, 0.1627124845981598, ('डॉक्टर', 'परिचारिका'))
(4.0, 0.2502923011779785, ('प्राध्यापक', 'डॉक्टर'))
(6.0, 0.36071258783340454, ('विद्यार्थी', 'प्राध्यापक'))
(4.375, 0.16025222837924957, ('स्मार्ट', 'विद्यार्थी'))
(3.0, 0.27255865931510925, ('स्मार्ट', 'मूर्ख'))
(6.375, 0.27047473192214966, ('पुस्तक', 'ग्रंथालय'))
(8.5, 0.2592158317565918, ('बँक', 'पैसा'))
(2.375, 0.1790100783109665, ('पवित्र', 'सेक्स'))
(6.375, 0.1945643573999405, ('टेनिस', 'रॅकेट'))
(8.375, 0.09642305225133896, ('अराफात', 'दहशतवादी'))
(0.0, 0.16742481291294098, ('अराफात', 'जॅक्सन'))
(8.5, 0.12597201764583588, ('कायदा', 'वकील'))
(8.0, 0.13652241230010986, ('चित्रपट', 'स्टार'))
(8.0, 0.0

In [12]:
#Hindi cc 300 pretrained on mr cognates
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.cognates2.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

(8.5, 0.3290926516056061, ('संगणक', 'इंटरनेट'))
(6.75, 0.4093678891658783, ('विमान', 'कार'))
(6.625, 0.19273625314235687, ('रेल्वे', 'कार'))
(9.0, 0.14239197969436646, ('टेलिफोन', 'संवाद'))
(7.0, 0.4051612317562103, ('दूरदर्शन', 'रेडिओ'))
(8.0, 0.3028254806995392, ('मीडिया', 'रेडिओ'))
(6.375, 0.1627124845981598, ('डॉक्टर', 'परिचारिका'))
(4.0, 0.2502923011779785, ('प्राध्यापक', 'डॉक्टर'))
(6.0, 0.36071258783340454, ('विद्यार्थी', 'प्राध्यापक'))
(4.375, 0.16025222837924957, ('स्मार्ट', 'विद्यार्थी'))
(3.0, 0.27255865931510925, ('स्मार्ट', 'मूर्ख'))
(6.375, 0.27047473192214966, ('पुस्तक', 'ग्रंथालय'))
(8.5, 0.2592158317565918, ('बँक', 'पैसा'))
(2.375, 0.1790100783109665, ('पवित्र', 'सेक्स'))
(6.375, 0.1945643573999405, ('टेनिस', 'रॅकेट'))
(8.375, 0.09642305225133896, ('अराफात', 'दहशतवादी'))
(0.0, 0.16742481291294098, ('अराफात', 'जॅक्सन'))
(8.5, 0.12597201764583588, ('कायदा', 'वकील'))
(1.0, 0.08874603360891342, ('पेय', 'गाडी'))
(0.0, 0.06241116672754288, ('पेय', 'कान'))
(6.5, 0.17306728661

In [38]:
! wc -l evaluation/mr.cognates.txt

      54 evaluation/mr.cognates.txt


In [53]:
#Marathi fasttext 300 on segmented data mr cognates
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.cognates.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
# print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

SpearmanrResult(correlation=0.36151091476459857, pvalue=0.006691104252438456)


In [9]:
#Hindi evaluate on Hindi test CROSSLINGUAL 300, using cc 300 pretrained to train
word_pairs, human_judgments = get_word_sim_dataset("evaluation/hin.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
# print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

SpearmanrResult(correlation=0.49907948569529287, pvalue=3.317890014380028e-16)


In [15]:
#Marathi CROSSLINGUAL 300, using cc 300 Hindi pretrained, 50k Marathi segmented to train
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(len(human_judgments))
# print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

63
SpearmanrResult(correlation=0.5556324073180827, pvalue=2.278879881999178e-06)


In [22]:
#Marathi CROSSLINGUAL 300, using cc 300 Hindi pretrained, 50k Marathi segmented to train
word_pairs, human_judgments = get_word_sim_dataset("evaluation/mr.word_sim.txt")
model_judgments = get_model_judgments(word_pairs)
human_judgments = [human_judgments[idx] for idx in range(len(human_judgments)) if model_judgments[idx] is not None]
model_judgments = list(filter(lambda x: x is not None, model_judgments))
print(len(human_judgments))
# print(*list(zip(human_judgments, model_judgments, word_pairs)), sep="\n")
spearman_correlation = stats.spearmanr(model_judgments, human_judgments)
print(spearman_correlation)

कळफलक
टेलिफोन
दूरदर्शन
बटाटा
अंडी
कोबी
यासिर
शांतता
अराफात
दहशतवादी
अराफात
पॉपकॉर्न
भौतिकशास्त्र
प्रोटॉन
रसायनशास्त्र
भौतिकशास्त्र
रसायनशास्त्र
रत्नजडित
शेगडी
कोंबडा
कोंबडा
स्मशानभूमी
काच
जादूगार
प्राणीसंग्रहालय
मानसशास्त्र
मनोदोषचिकित्सा
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
मानसशास्त्र
आकाशगंगा
खगोलशास्त्रज्ञ
ऑब्जेक्ट
जग्वार
जग्वार
प्रयोगशाळा
प्रयोगशाळा
लँडस्केप
सैन्याने
जपानी
104
SpearmanrResult(correlation=0.39104227109002043, pvalue=4.054736400322689e-05)
